<center><b><font size="+2"> SISTEMA DE RECOMENDAÇÃO </font></b>


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import matplotlib.pyplot as plt
from ast import literal_eval
import nltk
from nltk.stem import SnowballStemmer

df_movies = pd.read_csv(r'C:/Users/juliana/imbd_movies/movies_metadata.csv')
df_credits = pd.read_csv(r'C:/Users/juliana/imbd_movies/credits.csv')
df_keywords = pd.read_csv(r'C:/Users/juliana/imbd_movies/keywords.csv')
links_small = pd.read_csv(r'C:/Users/juliana/imbd_movies/links_small.csv')
ratings_small = pd.read_csv(r'C:/Users/juliana/imbd_movies/ratings_small.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
def removeTextRows(df, columnName):
    index_0=[]
    
    for i in df.itertuples():
        myValue = getattr(i, columnName)
        if(str.isdigit(myValue)):
            index_0.append(i.Index)

    df=df.loc[index_0] 
    df.reset_index(level=0, inplace=True)
    return df   

def weightedRating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

def getRecommendations(title, cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df_movies['title'].iloc[movie_indices]

def getDirector(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def filterKeywords(y):
    words = []
    for i in x:
        if i in y:
            words.append(i)
    return words

In [3]:
df_movies['id'] = df_movies['id'].astype('string')
df_movies['budget'] = df_movies['budget'].astype('string')
df_movies = removeTextRows(df_movies,'budget')
df_movies['budget']=pd.to_numeric(df_movies['budget'])
df_movies['popularity']=pd.to_numeric(df_movies['popularity'])

df_credits['id'] = df_credits['id'].astype('string')
df_movies = df_movies.merge(df_credits, on='id')


In [4]:
df_movies = df_movies[:5000]

<h3> Recomendação baseada na descrição do filme </h3>

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
df_movies['overview'] = df_movies['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(df_movies['overview'])
tfidf_matrix.shape

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df_movies.index, index=df_movies['title']).drop_duplicates()


In [6]:
getRecommendations('Taxi Driver',cosine_sim)

2593                My Son the Fanatic
1851                   Midnight Cowboy
3292    We're Back! A Dinosaur's Story
4364                      Biloxi Blues
3367                    Jacob's Ladder
1469                 The Fifth Element
4821                  The Exterminator
335                            The War
2076                       Family Plot
3333                  Band of the Hand
Name: title, dtype: object

<h3> Recomendação baseada nos metadados (palavras-chaves, elenco e diretor)</h3>


In [7]:
df_keywords['id'] = df_keywords['id'].astype('string')

df_movies = df_movies.merge(df_keywords, on='id')
links_small.columns =['movieid','imdbId','id']
links_small.dropna(inplace=True)
links_small['id'] = links_small['id'].astype('int')
links_small['id'] = links_small['id'].astype('string')

df_linksmall_movies = df_movies.merge(links_small, on='id',how = 'inner')
df_linksmall_movies['cast'] = df_linksmall_movies['cast'].apply(literal_eval)
df_linksmall_movies['crew'] = df_linksmall_movies['crew'].apply(literal_eval)
df_linksmall_movies['keywords'] = df_linksmall_movies['keywords'].apply(literal_eval)
df_linksmall_movies['cast_size'] = df_linksmall_movies['cast'].apply(lambda x: len(x))
df_linksmall_movies['crew_size'] = df_linksmall_movies['crew'].apply(lambda x: len(x))

df_linksmall_movies['cast'] = df_linksmall_movies['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_linksmall_movies['cast'] = df_linksmall_movies['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
df_linksmall_movies['cast'] = df_linksmall_movies['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

df_linksmall_movies['keywords'] = df_linksmall_movies['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

df_linksmall_movies['director'] = df_linksmall_movies['crew'].apply(getDirector)
df_linksmall_movies['director'] = df_linksmall_movies['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
df_linksmall_movies['director'] = df_linksmall_movies['director'].apply(lambda x: [x,x, x])

y = df_linksmall_movies.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
y.name = 'keyword'

y = y.value_counts()
y = y[y > 1]

stemmer = nltk.stem.SnowballStemmer('english')

def filterKeywords(x):
    words = []
    for i in x:
        if i in y:
            words.append(i)
    return words

df_linksmall_movies['keywords'] = df_linksmall_movies['keywords'].apply(filterKeywords)
df_linksmall_movies['keywords'] = df_linksmall_movies['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
df_linksmall_movies['keywords'] = df_linksmall_movies['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
df_linksmall_movies['soup'] = df_linksmall_movies['keywords'] + df_linksmall_movies['cast'] + df_linksmall_movies['director']
df_linksmall_movies['soup'] = df_linksmall_movies['soup'].apply(lambda x: ' '.join(x))

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df_linksmall_movies['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)

df_linksmall_movies = df_linksmall_movies.reset_index()
titles = df_linksmall_movies['title']
indices = pd.Series(df_linksmall_movies.index, index=df_linksmall_movies['title'])

C:\Users\JULIAN~1.PER\AppData\Local\Temp/ipykernel_8928/2974363026.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y = df_linksmall_movies.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


In [8]:
getRecommendations('Star Wars',cosine_sim)

2120               Young and Innocent
970                A Farewell to Arms
2705    An American Werewolf in Paris
983                      Maximum Risk
1082                        The Doors
2452           Relax... It's Just Sex
1360                           Jaws 2
1480                     Fathers' Day
2467                         Ravenous
3935                    Thirteen Days
Name: title, dtype: object

<h3> Recomendação por filtragem colaborativa </h3>

A filtragem colaborativa leva em conta as avaliações aos filmes de outros usuários com gostos similares

In [9]:
def calculateUserSimilarity(ratings_matrix):
    sim_matrix = pd.DataFrame(index=ratings_matrix.index, columns=ratings_matrix.index)
    for i in ratings_matrix.index:
        for j in ratings_matrix.index:
            sim_matrix.loc[i,j] = 1 - (ratings_matrix.loc[i] - ratings_matrix.loc[j]).abs().sum()/(ratings_matrix.loc[i].count())
    return sim_matrix

def getUserRecommendations(user_id, ratings_matrix, sim_matrix):
    similar_users = sim_matrix.loc[user_id].sort_values(ascending=False)[1:]
    user_ratings = ratings_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings==0].index
    item_scores = []
    for item in unrated_items:
        score = 0
        for sim_user, sim_score in similar_users.items():
            if ratings_matrix.loc[sim_user,item] != 0:
                score += sim_score * ratings_matrix.loc[sim_user,item]
        item_scores.append((item, score))
    recommendations = sorted(item_scores, key=lambda x:x[1], reverse=True)[:10]
    return recommendations

In [10]:
df_ratings = ratings_small[:3000]
ratings_matrix = df_ratings.pivot_table(index=['userId'], columns=['movieId'], values='rating')
ratings_matrix = ratings_matrix.fillna(0)
similarity_matrix = calculateUserSimilarity(ratings_matrix)

In [11]:
user_id=14
recommendations = getUserRecommendations(user_id, ratings_matrix, similarity_matrix)
print(f"Recomendações para o usuário {user_id}:")
for item_id, score in recommendations:
    print(f"Item {item_id}: {score}")

Recomendações para o usuário 14:
Item 318: 26.686914736323317
Item 356: 22.531912272055205
Item 2762: 18.75431246919665
Item 527: 18.23533760473139
Item 296: 16.646500739280434
Item 50: 16.177550517496307
Item 1197: 15.923607688516512
Item 110: 13.971414489896501
Item 1220: 13.553474618038443
Item 588: 13.37678659438147
